### * RNN 주요 레이어 종류
#### (1) SimpleRNN :가장 간단한 형태의 RNN레이어, 활성화 함수로 tanh가 사용됨(tanh: -1 ~ 1 사이의 값을 반환)
#### (2) LSTM(Long short Term Memory) : 입력 데이터와 출력 사이의 거리가 멀어질수로 연관 관계가 적어진다(Long Term Dependency,장기의존성 문제), LSTM은 장기 의존성 문제를 해결하기 위해 출력값외에 셀상태(cell state)값을 출력함, 활성화 함수로 tanh외에 sigmoid가 사용됨
#### (3) GRU(Gated Recurent Unit) : 뉴욕대 조경현 교수 등이 제안, LSTM보다 구조가 간단하고 성능이 우수함

In [1]:
import tensorflow as tf
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

print(tf.__version__)

2.17.0


In [2]:
# Sequence data
X = np.array([[0,1,2,3],
              [1,2,3,4],
              [2,3,4,5],
              [3,4,5,6],
              [4,5,6,7],
              [5,6,7,8]],dtype=np.float32)

x_data = tf.reshape(X,(-1,4,1))  # (6,4,1)  (N,T,D)

y_data = np.array([4,5,6,7,8,9],dtype=np.float32)

print(x_data.shape,y_data.shape)
# print(type(x_data),type(y_data))
x_data

(6, 4, 1) (6,)


<tf.Tensor: shape=(6, 4, 1), dtype=float32, numpy=
array([[[0.],
        [1.],
        [2.],
        [3.]],

       [[1.],
        [2.],
        [3.],
        [4.]],

       [[2.],
        [3.],
        [4.],
        [5.]],

       [[3.],
        [4.],
        [5.],
        [6.]],

       [[4.],
        [5.],
        [6.],
        [7.]],

       [[5.],
        [6.],
        [7.],
        [8.]]], dtype=float32)>

### [1] SimpleRNN
#### 가장 간단한 형태의 RNN 

In [3]:
# RNN 순환 신경망 구현  : SimpleRNN
model = tf.keras.Sequential([
    # X: (N,D) , Wx:(D,H), Wh : (H,H), b: H
    #    (6,1)    (1,300)    (300,300)  300  --> 1*300+300*300+300 = 90600개의 Param
    # (N,T,D) : (6,4,1)  ---> (N,T,H) : (6,4,300) , T는 sequence_length, H는 Hidden Size
    # return_sequences=True는 3차원(N,T,H)로 출력 
    # return_sequences=False는 2차원(N,H)로 출력 ,기본 값

    tf.keras.layers.SimpleRNN(units=300,return_sequences=True,input_shape=(4,1)),
    tf.keras.layers.SimpleRNN(units=300),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam',loss='mse')
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)               │ (None, 4, 300)              │          90,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_1 (SimpleRNN)             │ (None, 300)                 │         180,300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             301 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 271,201 (1.03 MB)

 Trainable params: 271,201 (1.03 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
# 학습 및 예측
model.fit(x_data,y_data,epochs=100)
print(model.predict(x_data))

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 55.0424
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 2.4419
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 4.5199
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 7.4131
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 4.2750
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 1.5384
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 2.2051
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 3.6567
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 3.3211
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 2.0065
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 1.0815
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.9115
Epoch 13/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 1.1599
Epoch 14/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 1.3866
Epoch 15/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 1.3480
Epoch 16/100
1/1 ━━━

In [5]:
y_data

array([4., 5., 6., 7., 8., 9.], dtype=float32)

In [5]:
# 학습되지 않은 입력 데이터에 대한 예측 결과
print(model.predict(np.array([[[6.],[7.],[8.],[9.]]])))
print(model.predict(np.array([[[-1.],[0.],[1.],[2.]]])))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 307ms/step
[[9.46961]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
[[1.4189062]]


In [6]:
# 평가
model.evaluate(x_data, y_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 439ms/step - loss: 0.0053


0.005293408874422312

### [2] LSTM(Long short Term Memory)
#### 입력 데이터와 출력 사이의 거리가 멀어질수로 연관 관계가 적어진다(Long Term Dependency,장기의존성 문제)
#### LSTM은 장기 의존성 문제를 해결하기 위해 출력값외에 셀상태(cell state)값을 출력함

In [8]:
# RNN 순환 신경망 구현  : LSTM
model = tf.keras.Sequential([
    # X: (N,D) , Wx:(D,4H), Wh : (H,4H), b: 4H
    #    (6,1)    (1,4*300)   (300,4*300)  4*300  --> 1*4*300+300*4*300+4*300 = 4*90600개의 Param
    # (N,T,D) : (6,4,1)  ---> (N,T,H) : (6,4,300) , T는 sequence_length, H는 Hidden Size
    
    tf.keras.layers.LSTM(units=300, return_sequences=True, input_shape=[4,1]),
    tf.keras.layers.LSTM(units=300),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                        │ (None, 4, 300)              │         362,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 300)                 │         721,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │             301 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,083,901 (4.13 MB)

 Trainable params: 1,083,901 (4.13 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# 학습 및 예측
model.fit(x_data,y_data,epochs=100,verbose=0)
print(model.predict(x_data))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 352ms/step
[[3.3923886]
 [5.0921965]
 [6.36059  ]
 [7.296271 ]
 [7.9968348]
 [8.530707 ]]


### [3] GRU(Gated Recurent Unit)
#### 뉴욕대 조경현 교수 등이 제안, LSTM보다 구조가 간단하고 성능이 우수

In [11]:
# RNN 순환 신경망 구현  : GRU
model = tf.keras.Sequential([
    tf.keras.layers.GRU(units=300, return_sequences=True, input_shape=[4,1]),
    tf.keras.layers.GRU(units=300),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_2 (GRU)                          │ (None, 4, 300)              │         272,700 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_3 (GRU)                          │ (None, 300)                 │         541,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1)                   │             301 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 814,801 (3.11 MB)

 Trainable params: 814,801 (3.11 MB)

 Non-trainable params: 0 (0.00 B)